<a href="https://colab.research.google.com/github/lucia1299/DBMS_InvestigativeJournalism/blob/main/NER_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pandas torch datasets seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 566.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install transformers[torch]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TESI/github/WN_test.tsv', sep='\t', names=["token", "label"], quoting=3)

In [ ]:
dataset = Dataset.from_pandas(test_df)

In [ ]:
dataset

Dataset({
    features: ['token', 'label'],
    num_rows: 56519
})

# UMBERTo

In [ ]:
from transformers import CamembertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import pandas as pd
import numpy as np
import datasets


tokenizer = CamembertTokenizerFast.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1")
model = AutoModelForTokenClassification.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1", num_labels=7)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/794k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at Musixmatch/umberto-commoncrawl-cased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
encoded_inputs = tokenizer(
    dataset['token'],
    max_length=512,
    truncation=True,
    padding=True
)

In [ ]:
len(dataset['label']), len(encoded_inputs['input_ids'])

(56519, 56519)

In [ ]:
# Extract all unique labels from your dataset
unique_labels = set()
for item in dataset:
    unique_labels.add(item['label'])

# Create a label mapping dictionary and relative function
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

def map_labels_to_ids(examples):
    examples['label'] = [label_mapping[label] for label in examples['label']]
    return examples

In [ ]:
label_list = list(unique_labels)
label_list

['I-PER', 'B-LOC', 'B-PER', 'I-ORG', 'I-LOC', 'B-ORG', 'O']

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(
        examples['token'],
        truncation=True,
        is_split_into_words=False,
        padding=True
    )

    labels = []
    for i, label in enumerate(examples['label']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_mapping.get(label, -100))
            else:
                label_ids.append(label_mapping.get(label, -100) if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Apply the label mapping
tokenized_dataset = tokenized_dataset.map(map_labels_to_ids, batched=True)

Map:   0%|          | 0/56519 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/56519 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # Flatten the predictions and labels to 1D arrays
    predictions = predictions.flatten()
    labels = labels.flatten()

    # Remove ignored indices (-100)
    valid_indices = labels != -100
    predictions = predictions[valid_indices]
    labels = labels[valid_indices]

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }


In [ ]:
# Check the lengths
print("Number of tokenized elements:", len(tokenized_dataset['input_ids']))
print("Number of labels:", len(tokenized_dataset['label']))

Number of tokenized elements: 56519
Number of labels: 56519


In [ ]:
#Check the type
print(type(tokenized_dataset['input_ids'][0][0]))
print(type(tokenized_dataset['label'][0]))

<class 'int'>
<class 'int'>


In [ ]:
# Define evaluation arguments
training_args = TrainingArguments(
    per_device_eval_batch_size=8,
    output_dir='./results',
    evaluation_strategy='epoch',
    save_steps=1000,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.79152250289917, 'eval_model_preparation_time': 0.0108, 'eval_accuracy': 0.6727259092613423, 'eval_f1': 0.703145346187599, 'eval_precision': 0.7384418653987673, 'eval_recall': 0.6727259092613423, 'eval_runtime': 95.9997, 'eval_samples_per_second': 588.741, 'eval_steps_per_second': 73.594}


# Bertolde

In [ ]:
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import pandas as pd
import numpy as np
from datasets import Dataset

tokenizer = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/Colab Notebooks/TESI/github/bertolde_model')
model = AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/TESI/github/bertolde_model')

In [ ]:
encoded_inputs = tokenizer(
    dataset['token'],
    max_length=512,
    truncation=True,
    padding=True
)

In [ ]:
len(dataset['label']), len(encoded_inputs['input_ids'])

(56519, 56519)

In [ ]:
# Function to map GPE labels to LOC
def map_gpe_to_loc(predictions):
    mapped_predictions = []
    for label in predictions:
        if label == 'B-GPE':
            mapped_predictions.append('B-LOC')
        elif label == 'I-GPE':
            mapped_predictions.append('I-LOC')
        else:
            mapped_predictions.append(label)
    return mapped_predictions

for item in dataset:
    item['label'] = map_gpe_to_loc(item['label'])

# Extract all unique labels from your dataset after mapping GPE to LOC
unique_labels = set()
for item in dataset:
    unique_labels.add(item['label'])

# Create a label mapping dictionary and relative function
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

def map_labels_to_ids(examples):
    examples['label'] = [label_mapping[label] for label in examples['label']]
    return examples

def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(
        examples['token'],
        truncation=True,
        is_split_into_words=False,
        padding=True
    )

    labels = []
    for i, label in enumerate(examples['label']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_mapping.get(label, -100))
            else:
                label_ids.append(label_mapping.get(label, -100) if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs


In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Apply the label mapping
tokenized_dataset = tokenized_dataset.map(map_labels_to_ids, batched=True)

Map:   0%|          | 0/56519 [00:00<?, ? examples/s]

Map:   0%|          | 0/56519 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # Flatten the predictions and labels to 1D arrays
    predictions = predictions.flatten()
    labels = labels.flatten()

    # Remove ignored indices (-100)
    valid_indices = labels != -100
    predictions = predictions[valid_indices]
    labels = labels[valid_indices]

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

In [ ]:
# Check the lengths
print("Number of tokenized sentences:", len(tokenized_dataset['input_ids']))
print("Number of labels:", len(tokenized_dataset['label']))

Number of tokenized sentences: 56519
Number of labels: 56519


In [ ]:
#Check the type
print(type(tokenized_dataset['input_ids'][0][0]))
print(type(tokenized_dataset['label'][0]))

<class 'int'>
<class 'int'>


In [ ]:
# Define evaluation arguments
training_args = TrainingArguments(
    per_device_eval_batch_size=8,
    output_dir='./results',
    evaluation_strategy='epoch'
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 9.664316177368164, 'eval_model_preparation_time': 0.0097, 'eval_accuracy': 0.03576911721287275, 'eval_f1': 0.023667502687601014, 'eval_precision': 0.14483782177606502, 'eval_recall': 0.03576911721287275, 'eval_runtime': 87.2613, 'eval_samples_per_second': 647.698, 'eval_steps_per_second': 80.964}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
